In [2]:
import numpy as np
import pandas as pd
import patsy
import sklearn.linear_model as linear_model

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV

#### Read in data and conduct an exploratory data analysis.
#### Resolve any data issues you identify and articulate why you did what you did.

In [3]:
sf_crime = pd.read_csv('datasets/sf_crime_train.csv')

#### Create column for hour, month, and year from 'Dates' column.
##### Hint: pd.to_datetime may be helpful.

In [4]:
# print sf_crime.head()
sf_crime['Dates'] = pd.to_datetime(sf_crime.Dates)

sf_crime['hour'] = sf_crime.Dates.dt.hour
sf_crime['month'] = sf_crime.Dates.dt.month
sf_crime['year'] = sf_crime.Dates.dt.year

### Build a logit model predicting violent crime versus non-violent crime versus non-crimes.

##### We've given you the various crimes that fall into specific categories so you can use these to create these subcategories 

#### Non-Violent Crimes: bad checks, bribery, drug/narcotic, drunkenness, embezzlement, forgery/counterfeiting, fraud, gambling, liquor, loitering, trespass.

#### Non-Crimes: non-criminal, runaway, secondary codes, suspicious occ, warrants.

#### Violent Crimes: everything else.

###### Hint: What type of model do you need here? What should your "baseline" category be?

Need a logistic regression model
baseline = violent crime

#### Fit a model with five folds and lasso regularization
#### Use Cs=15 to test a grid of 15 distinct parameters
#### Remember: Cs describes the inverse of regularization strength

In [10]:
nvc = ['BAD CHECKS', 'BRIBERY', 'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING', 'LIQUOR', 'LOITERING', 'TRESSPASS']
nc = ['NON-CRIMINAL', 'RUNAWAY', 'SECONDARY CODES', 'SUSPICIOUS OCC', 'WARRANTS']

sf_crime['Subcategory'] = ['Non-Violent Crime' if x in (nvc) else 'Non-Crime' if x in (nc) else 'Violent Crime' for x in sf_crime['Category']]

X = sf_crime['Subcategory'!='Violent Crime']
y = sf_crime['Subcategory'=='Violent Crime']

logreg_cv = LogisticRegressionCV(solver='liblinear', Cs=15, cv=5, penalty='l1')
logreg_cv.fit(X, y)
print ('Max auc_roc:', logreg_cv.scores_[1].max())



# C = [{ 'C' : [1,0.1,0.01,0.001,0.0001,0] },
#      { 'C' : [1,0.1,0.01,0.001,0.0001,0] } ]

# logreg = LogisticRegression()
# C_vals = [0.0001, 0.001, 0.01, 0.1, 0.5, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]
# penalties = ['l1','l2']

# alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
# grid = GridSearchCV(estimator=logreg_cv, cv=5, param_grid=dict(alpha=C_vals))
# grid.fit(sf_crime, sf_crime['vc'])
# print(grid)


KeyError: True

In [23]:
# find best C per class
print('best C for class:')
best_C = {logreg_cv.classes_[i]:x for i, (x, c) in enumerate(zip(logreg_cv.C_, logreg_cv.classes_))}
print(best_C)

best C for class:


AttributeError: 'LogisticRegressionCV' object has no attribute 'C_'

In [52]:
LogisticRegressionCV.get_params().keys()

TypeError: unbound method get_params() must be called with LogisticRegressionCV instance as first argument (got nothing instead)

#### Fit regular logit model to 'DRUG/NARCOTIC' and 'BURGLARY' classes; be sure to specify the Y = 1 and Y = 0 classes. Use lasso penalty

#### Build confusion matrices for the models above


#### Run gridsearch using GridSearchCV and 5 folds
#### Score on accuracy; what does this metric tell us?

In [ ]:
logreg = LogisticRegression()
C_vals = [0.0001, 0.001, 0.01, 0.1, 0.5, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]
penalties = ['l1','l2']

#### Find the best parameter

#### Use this parameter to .fit, .predict, and print a classification_report (we've already imported this for you) for our X and Y

#### When looking at the classification report, remember:
##### Precision = True Positives / (True Positives + False Positives)

A precision score of 1 indicates that the classifier never mistakenly added observations from another class. A precision score of 0 would mean that the classifier misclassified every instance of the current class.
 
##### Recall = True Positives / (True Positives + False Negatives)
A recall score of 1 indicates that the classifier correctly predicted (found) all observations of the current class (by implication, no false negatives, or misclassifications of the current class). A recall score of 0 alternatively means that the classifier missed all observations of the current class.

##### F1-Score = 2 * (Precision * Recall) / (Precision + Recall)
The f1-score's best value is 1 and worst value is 0, like the precision and recall scores. It is a useful metric for taking into account both measures at once. 

##### Support is simply the number of observations of the labelled class.

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html